In [1]:
from dataclasses import dataclass
import numpy as np
import torch
import math
from botorch.generation import MaxPosteriorSampling
from torch.quasirandom import SobolEngine
import botorch
import gpytorch

In this notebook, we will see how to use the TuRBO Bayesian Optimization tool for the capstone project. TuRBO is a BO algorithm proposed by Uber that specializes in high-dimensional problems. You can read the details of the algorithm in the paper:


Eriksson et al., "Scalable Global Optimization via Local Bayesian Optimization", NeurIPS (2019). URL: https://proceedings.neurips.cc/paper/2019/file/6c990b7aca7bc7058f5e98ea909e924b-Paper.pdf

For implementing the method, we will be using the Gaussian Process library GPyTorch, and the Bayesian Optimization library BoTorch. We will be loosely following the tutorial made by BoTorch's team:

https://botorch.org/tutorials/turbo_1

However, we will be making some modification that are case-specific for us.

TuRBO works by creating a Trust Region over which will focus all our optimization efforts. This works great for higher-dimensions because the search space is too large and algorithms tend to over-explore! 

We keep track of a 'Turbo State' that dictates the size and location of the region. The code below implements a data class that will help us keep track of the state:

In [2]:
# we define a dataclass for our state
@dataclass
class TurboState:
    dim: int # dimension of the problem, aka input dimension
    batch_size: int = 1 # we could do batch optimization, but the capstone only does one query at a time
    length: float = 0.8 # the length of the current trust region
    length_min: float = 0.5 ** 7 # minimum length for the trust region
    length_max: float = 1.6 # maximum length for the trust region
    failure_counter: int = 0 # initialize counter of the number of failures to improve on the best observation
    failure_tolerance: int = float("nan")  # Note: Post-initialized
    success_counter: int = 0 # initialize counter of the number of success to improve on the best observation
    success_tolerance: int = 10  # Note: The original paper uses 3, this is the number of successes in a row needed to expand the region
    best_value: float = -float("inf") # best value so far, initialized to be the infimum
    restart_triggered: bool = False 

    def __post_init__(self): # sets the failure tolerance based on the problem's dimension and batch size.
        self.failure_tolerance = math.ceil(
            max([4.0 / self.batch_size, float(self.dim) / self.batch_size]) # number of failures needed in a row to shrink the trust region
        )


def update_state(state, Y_next):  # updates the TurboState based on new observations (Y_next). 
    #It checks if the new observation is a success or a failure and accordingly updates the success and failure counters. 
    #It also adjusts the trust region's length based on these counters.
    # count if a success, otherwise a failure
    if max(Y_next) > state.best_value + 1e-3 * math.fabs(state.best_value):
        state.success_counter += 1
        state.failure_counter = 0
    else:
        state.success_counter = 0
        state.failure_counter += 1
    # check if we need to expand or shrink the trust region
    if state.success_counter == state.success_tolerance:  # Expand trust region
        state.length = min(2.0 * state.length, state.length_max)
        state.success_counter = 0
    elif state.failure_counter == state.failure_tolerance:  # Shrink trust region
        state.length /= 2.0
        state.failure_counter = 0
    # set the best value if we got a new observation
    state.best_value = max(state.best_value, max(Y_next).item())
    if state.length < state.length_min:
        state.restart_triggered = True
    return state

It will be very important to keep track of the state when we optimize, as we will need to make sure we keep the state updated from query to query. You can use a print statement to see the value of a state:

In [3]:
state = TurboState(dim = 6)
print(state)

TurboState(dim=6, batch_size=1, length=0.8, length_min=0.0078125, length_max=1.6, failure_counter=0, failure_tolerance=6, success_counter=0, success_tolerance=10, best_value=-inf, restart_triggered=False)


It is important to record these variables after choosing a new query, and re-input, and update to the correct state when we receive new observations. An example of this will be given later. We can then define the TuRBO loop:

In [4]:
def generate_batch(
    #selecting the next point for evaluation in Bayesian optimization allows for balancing exploration (trying out new areas of the search space) and exploitation (focusing on areas known to be promising). 
    #The use of a GP model enables the method to incorporate prior knowledge and the results of past evaluations to make informed decisions about where to search next.
    #generates the next query point in the optimization process using Thompson Sampling (TS). 
    #It requires a GP model and training data as inputs and operates within the current trust region.
    state,
    model,  # GP model
    X,  # Evaluated points on the domain [0, 1]^d
    Y,  # Function values
    batch_size = 1, # fix batch size to 1
    n_candidates=None,  # Number of candidates for Thompson sampling
    num_restarts=10,
    raw_samples=512,
    acqf="ts",  # "ei" or "ts"
):
    assert acqf in ("ts")
    assert X.min() >= 0.0 and X.max() <= 1.0 and torch.all(torch.isfinite(Y))
    if n_candidates is None:
        n_candidates = min(5000, max(2000, 200 * X.shape[-1]))

    # Scale the trust region to be proportional to the lengthscales
    x_center = X[Y.argmax(), :].clone()
    weights = model.covar_module.base_kernel.lengthscale.squeeze().detach()
    weights = weights / weights.mean()
    weights = weights / torch.prod(weights.pow(1.0 / len(weights)))
    tr_lb = torch.clamp(x_center - weights * state.length / 2.0, 0.0, 1.0)
    tr_ub = torch.clamp(x_center + weights * state.length / 2.0, 0.0, 1.0)
    # we focus only on thompson sampling as an acquisition function
    if acqf == "ts":
        dim = X.shape[-1]
        sobol = SobolEngine(dim, scramble=True)
        pert = sobol.draw(n_candidates)
        pert = tr_lb + (tr_ub - tr_lb) * pert

        # Create a perturbation mask
        prob_perturb = min(20.0 / dim, 1.0)
        mask = (
            torch.rand(n_candidates, dim)
            <= prob_perturb
        )
        ind = torch.where(mask.sum(dim=1) == 0)[0]
        mask[ind, torch.randint(0, dim - 1, size=(len(ind),))] = 1

        # Create candidate points from the perturbations and the mask        
        X_cand = x_center.expand(n_candidates, dim).clone()
        X_cand[mask] = pert[mask]

        # Sample on the candidate points
        # set model to evaluation mode
        model.eval()
        posterior_distribution = model(X_cand)
        with torch.no_grad():  # We don't need gradients when using TS
            posterior_sample = posterior_distribution.sample()
            X_next_idx = torch.argmax(posterior_sample)
            X_next = X_cand[X_next_idx]

    return X_next

The function above requires us to use a GPyTorch model as an input. A tutorial on how GPyTorch models can be used is found here: https://docs.gpytorch.ai/en/stable/examples/01_Exact_GPs/Simple_GP_Regression.html

Below we define our model class:

In [5]:
# we use the model given in the tutorial, we also add the hyper-parameter training as a method
class ExactGPModel(gpytorch.models.ExactGP): #  defines a Gaussian Process model with a constant mean and an RBF kernel. 
    #It's used in the optimization process to model the objective function.
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        # set a constant mean
        self.mean_module = gpytorch.means.ConstantMean()
        # use a simple RBF kernel with constant scaling
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=train_x.shape[1]))
        # set number of hyper-parameter training iterations
        self.training_iter = 200

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

For the purposes of this notebook, we will optimize the function:

$$f(x_1, x_2, x_3, x_4, x_5, x_6) = x_3 * \sin(x_1) * \cos(x_2) + x_4 * x_5 - x_6 * x_5^2$$

We will create an initial data set at random.

Do not forget to re-define our state as we have a new best-observation!

In [6]:
# An example function is provided to illustrate the optimization process. It's a multi-dimensional function involving sine, cosine, and polynomial terms.
func = lambda x: torch.sin(x[:, 0]) * torch.cos(x[:, 1]) * x[:, 2] + x[:, 3] * x[:, 4] - x[:, 5] * (x[:, 4]**2)

train_x = torch.rand(size = torch.Size([15, 6]))
train_y = func(train_x)

state = TurboState(dim = 6, best_value = torch.max(train_y).float())

We now need to train the hyper-parameters of the model. This can be done in a similar fashion to a normal PyTorch model.

All we need is to define a model and a likelihood, and then activate .train() mode. We then follow classical PyTorch syntax:

In [7]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

#training the hyperparameters of the GP model with a likelihood function and an optimizer. 
#It also calculates the marginal log likelihood as a loss function.
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


for i in range(model.training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    if i % 10 == 9:
        print(f'Iter %d/%d - Loss: %.3f   lengthscale: {model.covar_module.base_kernel.lengthscale.detach()}   noise: %.3f' % (
            i + 1, model.training_iter, loss.item(),
            model.likelihood.noise.item()
        ))
    optimizer.step()

Iter 10/200 - Loss: 0.616   lengthscale: tensor([[1.2320, 1.2288, 1.2312, 1.2162, 1.2171, 1.2289]])   noise: 0.339
Iter 20/200 - Loss: 0.211   lengthscale: tensor([[1.9042, 1.9127, 1.9246, 1.6649, 1.6757, 1.8806]])   noise: 0.135
Iter 30/200 - Loss: -0.081   lengthscale: tensor([[2.4718, 2.5382, 2.5508, 1.3199, 1.3589, 2.3572]])   noise: 0.052
Iter 40/200 - Loss: -0.305   lengthscale: tensor([[2.8971, 3.0906, 3.0451, 0.5891, 0.6284, 2.6822]])   noise: 0.023
Iter 50/200 - Loss: -0.361   lengthscale: tensor([[3.1547, 3.4642, 3.2982, 0.3669, 0.4352, 2.9549]])   noise: 0.012
Iter 60/200 - Loss: -0.366   lengthscale: tensor([[3.1634, 3.5397, 3.1977, 0.4212, 0.4745, 2.9167]])   noise: 0.009
Iter 70/200 - Loss: -0.381   lengthscale: tensor([[2.8167, 3.3782, 2.7320, 0.4494, 0.5047, 2.5991]])   noise: 0.008
Iter 80/200 - Loss: -0.397   lengthscale: tensor([[2.0876, 3.1984, 2.1264, 0.5052, 0.5713, 2.4714]])   noise: 0.007
Iter 90/200 - Loss: -0.425   lengthscale: tensor([[1.2178, 3.2938, 1.6378,

We can now define a function that takes as input:
1. Training Data
2. A TuRBO State

And returns the next suggested query! We will define the GP model and optimize the GP's hyper-parameters inside the function itself.

In [8]:
def next_query_via_TurBO(train_x, train_y, turbo_state, verbose = False):  # combines all the previous elements to suggest the next query point in the optimization process. 
    #It trains a GP model on the current training data and uses the generate_batch function to propose the next point.
    # the verbose variable decides wether to print the hyper-parameter optimization details
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood)

    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # demonstrates how to iteratively query new points and update the training data and the TurboState. 
    # After each query, the state is updated with the new observation.
    for i in range(model.training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        if (i % 10 == 9) & verbose:
            print(f'Iter %d/%d - Loss: %.3f   lengthscale: {model.covar_module.base_kernel.lengthscale}   noise: %.3f' % (
                i + 1, model.training_iter, loss.item(),
                model.likelihood.noise.item()
            ))
        optimizer.step()
    
    return generate_batch(turbo_state, model = model, X = train_x, Y = train_y)

Finally, we can obtain a suggested query!

In [9]:
next_query = next_query_via_TurBO(train_x=train_x, train_y=train_y, turbo_state=state)
print(f'Next chose query: {next_query}')

Next chose query: tensor([0.8103, 0.4809, 0.4493, 0.9277, 0.9918, 0.8265])


C:\Users\jdche\AppData\Roaming\Python\Python310\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
C:\Users\jdche\AppData\Roaming\Python\Python310\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(


It is important to keep track of the state, and also update it when we receive new information. For example, the state that was used to choose the query can be printed:

In [10]:
print('State at latest optimization loop:')
print()
print(state)

State at latest optimization loop:

TurboState(dim=6, batch_size=1, length=0.8, length_min=0.0078125, length_max=1.6, failure_counter=0, failure_tolerance=6, success_counter=0, success_tolerance=10, best_value=tensor(0.7864), restart_triggered=False)


Now let's see what observation we would have gotten:

In [11]:
y_next = func(next_query.reshape(1, -1))
print(y_next)

tensor([0.3956])


Now that we have a new observation, it is vital to update the state before optimizing again!

In [12]:
new_state = update_state(state, y_next)

We can now optimize again to obtain the next point:

In [13]:
train_x = torch.concat((train_x, next_query.reshape(1, -1)), dim = 0)
train_y = torch.concat((train_y, y_next))

next_next_query = next_query_via_TurBO(train_x, train_y, new_state)
print(f'New query:', next_next_query)

New query: tensor([0.9784, 0.4101, 0.9316, 0.8540, 0.9870, 0.2342])


If you feel TuRBO would help you with the high-dimensional problems in the Capstone, give it a go! There a few questions that may lead to better performance:

1. Maybe you can constraint some of the GPs hyper-parameters for better behaviour?
2. How are you planning to initialize the Turbo State for the first time?

So take the code from this notebook and modify it to your liking!